## PRPM Scrapper

In [10]:
from lexicraft.scrapers.prpm_scraper import PRPMScraper
PRPMscrap = PRPMScraper()


word = 'cari'
result = PRPMscrap.findWordMetaData(word)
print(result)

#found smth can be preprocess if not done here, like there will have '\xa' character in the result string

{'word': 'cari', 'meanings': ['bapa berusaha untuk mendapat (menjumpai dan lain-lain) sesuatu', 'mn harta milik (pencarian)', 'pendapatan, hasil pekerjaan'], 'synonym': [], 'antonym': []}


## PERIBAHASA Scrapper

In [23]:
from lexicraft.scrapers.peri_scraper import PeriScraper
peri_scrap = PeriScraper()
proverbs_data_AM = peri_scrap.scrape_page(peri_scrap.AM_Url)
proverbs_data_NZ = peri_scrap.scrape_page(peri_scrap.NZ_Url)
combined_peri = proverbs_data_AM + proverbs_data_NZ

for peri in combined_peri:
    proverb = peri.get("peri", "")
    meaning = peri.get("meaning", "")
    print(f"Peribahasa: {proverb} || Maksud: {meaning}")


KeyboardInterrupt



## Data Collection For Berita Harian

In [24]:
from lexicraft.scrapers.bharian_scraper import BHarianScraper
from lexicraft.util.kafka import KafkaStreamProducer

from pyspark.sql import SparkSession, Row
from datetime import datetime, timedelta

spark = SparkSession.builder.appName("DE-prj").getOrCreate()
bharianScraper = BHarianScraper()
# print("Scraping article links...")

crawlledUrls = bharianScraper.scrapArticleLinks(datetime(2024,12,22,12,0,0),'/berita/nasional')
if len(crawlledUrls) == 0:
    print("No Article Found")
if crawlledUrls is None:
    print('failed to get article links')

demo_num_of_article = -1
# demo purpose
if demo_num_of_article != -1:
    crawlledUrls = crawlledUrls[:demo_num_of_article]
    print(f"Current is in demo stage, {demo_num_of_article} number of article(s) will be scrapped")
    
url_RDD = spark.sparkContext.parallelize(crawlledUrls)
url_RDD = url_RDD.repartition(3)
scrappedData = url_RDD.mapPartitions(BHarianScraper.scrapBatchArticle).collect()
scrappedData = [data for data in scrappedData if data is not None]
scrapped_dict_list = []
for data in scrappedData:
    if isinstance(data, Row):  
        dict_data = data.asDict(recursive = True)
        scrapped_dict_list.append(dict_data)

kafka_producer = KafkaStreamProducer(topic_name="beritaH")
kafka_producer.send_data(scrapped_dict_list)
        
print(scrapped_dict_list)

24/12/22 15:45:49 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Links crawl to datetime  2024-12-22 15:45:50.179054
Links Crawling Starts...
Article teaser found on page 0.
Current no. of link: 10


[Stage 5:>                                                          (0 + 3) / 3]/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  sel

Data sent to topic beritaH
[{'title': 'Graduan OKU penglihatan mahu jadi inspirasi, pemangkin semangat golongan senasib', 'authorDetails': {'authName': 'Amir Abd Hamid', 'relLink': '/authors/amir-abd-hamid', 'email': 'amir_hamid@mediaprima.com.my'}, 'publishTime': 'Disember 22, 2024 @ 3:16pm', 'articleBrief': 'Graduan OKU penglihatan, Mohammad Faiz menerima sijil daripada Pengerusi Majlis Agama Islam Wilayah Persekutuan, Syed Kamarulzaman Syed Kabeer (kanan) di Majlis Graduasi dan Apresiasi Latihan Dan Kerjaya MAIWP Kali Ke-3 di Rumah Puspanita. - Foto FATHIL ASRI.', 'contents': ['KUALA LUMPUR: Diiktiraf graduan terbaik dalam kategori orang kelainan upaya (OKU) penglihatan bukan sahaja menjadi kebanggaan, tetapi pemangkin semangat buat insan istimewa, Mohammad Faiz Asangku Abdullah.', 'Mohammad Faiz, 50, berkata kejayaan itu mencetuskan keinginannya untuk terus melakar kejayaan dalam kerjaya sebagai pengamal refleksologi dan terapi, selain mendidik lebih ramai OKU penglihatan dalam bid